In [3]:
ticker = input('Please input the ticker of the stock: ')

In [5]:
def analysis(ticker):
    import requests
    from bs4 import BeautifulSoup
    import pandas as pd
    link = 'https://finance.yahoo.com/quote/' + ticker + '/analysis?p' + ticker
    response = requests.get(link)
    if response.status_code == 200:
        pass
    else:
        print('An error occurred when accessing the page!')
    result_page = BeautifulSoup(response.content,'lxml')
    df_list = []
    for table_tag in result_page.find_all('table'):
        col,data_col,data_frame = [], [], []
        for th_tag in table_tag.find_all('th'):
            col.append(th_tag.get_text())
        i = 1
        for td_tag in table_tag.find_all('td'):
            if 1 <= i <= 5:
                data_col.append(td_tag.get_text())
                i += 1
            else:
                data_frame.append(data_col)
                data_col = []
                i = 1
                data_col.append(td_tag.get_text())
                i += 1
        data_frame.append(data_col)
        df = pd.DataFrame(data_frame,columns = col)
        df.set_index(col[0],inplace=True)
        df_list.append(df)
    return df_list[0]
#     return df_list[0],df_list[1],df_list[2],df_list[3],df_list[4],df_list[5]

In [10]:
def holders(ticker):
    import requests
    from bs4 import BeautifulSoup
    import pandas as pd
    link = 'https://finance.yahoo.com/quote/' + ticker + '/holders?p' + ticker
    response = requests.get(link)
    result_page = BeautifulSoup(response.content,'lxml')
    df_list = []
    for div_tag in result_page.find_all('div'):
        if div_tag.get('class') in [['Mt(25px)', 'Ovx(a)', 'W(100%)'],['W(100%)', 'Mb(20px)']]:
            if div_tag.find('span').get_text() == 'Breakdown':
                j = 2
                col = ['Major Holders Breakdown',' ']
            else:
                j = 5
                col = []
            i = 1
            data_col,data_frame = [],[]
            for th_tag in div_tag.find_all('th'):
                col.append(th_tag.get_text())
            for td_tag in div_tag.find_all('td'):
                if 1 <= i <= j:
                    data_col.append(td_tag.get_text())
                    i += 1
                else:
                    data_frame.append(data_col)
                    data_col = []
                    i = 1
                    data_col.append(td_tag.get_text())
                    i += 1
            data_frame.append(data_col)
            df = pd.DataFrame(data_frame,columns = col)
            df.set_index(col[0],inplace=True)
            df_list.append(df)
    return df_list[0]
    #     print(tag.get('class'))

In [12]:
def get_financial(ticker,IS=0,BS=0,CF=0):
    import requests
    from bs4 import BeautifulSoup
    import pandas as pd
    IS_keyword = ['Revenue','Operating Expenses','Income from Continuing Operations','Non-recurring Events','Net Income']
    BS_keyword = ['Assets','Liabilities','Equity']
    CF_keyword = ['Net Income','Operating Activities','Investing Activities','Financing Activities','Exchange Rate Changes']
    link = 'https://finance.yahoo.com/quote/' + ticker + '/financials?p=' + ticker
    keyword = IS_keyword
    if BS:
        link = 'https://finance.yahoo.com/quote/' + ticker + '/balance-sheet?p=' + ticker
        keyword = BS_keyword
    if CF:
        link = 'https://finance.yahoo.com/quote/' + ticker + '/cash-flow?p=' + ticker
        keyword = CF_keyword
    response = requests.get(link)
    result_page = BeautifulSoup(response.content,'lxml')
    df_list = []
    col,data_col,data_frame = [None], [], []
    i = 1
    end = 0
    for tr_tag in result_page.find_all('tr'):
        for td_tag in tr_tag.find_all('td'):
            if td_tag.get('class') == ['C($gray)', 'Ta(end)']:
                col.append(td_tag.get_text())
            if not td_tag.get('class') in [['Fw(b)', 'Fz(15px)'],['C($gray)', 'Ta(end)'],['Fw(b)', 'Fz(15px)', 'Pb(8px)', 'Pt(36px)']]:
                if 1 <= i <= 5:
                    data_col.append(td_tag.get_text())
                    i += 1
                else:
                    data_frame.append(data_col)
                    data_col = []
                    i = 1
                    data_col.append(td_tag.get_text())
                    i += 1
            if tr_tag.get('class') == ['Bdbw(0px)!', 'H(36px)']:
                end += 1
            if end == 5:
                data_frame.append(data_col)
                df = pd.DataFrame(data_frame)
                df_list.append(df)
                end = 0
                i = 1
                data_col,data_frame = [], []
    for k in range(len(keyword)):
        col_name = col
        col_name[0] = keyword[k] + ' (All numbers in thousands)'
        df_list[k].columns = col_name
        df_list[k].set_index(col_name[0],inplace=True)
    return df_list

In [13]:
get_financial(ticker,BS=0)

[                                    12/31/2017  12/31/2016  12/31/2015  \
 Revenue (All numbers in thousands)                                       
 Total Revenue                       37,945,000  34,631,000  35,226,000   
 Cost of Revenue                      3,884,000   3,707,000   3,659,000   
 Gross Profit                        34,061,000  30,924,000  31,567,000   
 
                                     12/31/2014  
 Revenue (All numbers in thousands)              
 Total Revenue                       33,977,000  
 Cost of Revenue                      3,441,000  
 Gross Profit                        30,536,000  ,
                                                12/31/2017  12/31/2016  \
 Operating Expenses (All numbers in thousands)                           
 Research Development                                    -           -   
 Selling General and Administrative             21,273,000  19,901,000   
 Non Recurring                                           -           -   
 O